In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from gazpacho import get, Soup

In [2]:
tokenizer= AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model=AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")


In [3]:
tokens =tokenizer.encode("It was good but ",return_tensors='pt')
tokens

tensor([[  101, 10197, 10140, 12050, 10502,   102]])

In [4]:
tokenizer.decode(tokens[0])
    
    

'[CLS] it was good but [SEP]'

In [5]:
result=model(tokens)
result
# The 1-5 sentiment ratings in the logits

SequenceClassifierOutput(loss=None, logits=tensor([[-1.4108,  0.0702,  1.6011,  0.9514, -1.0251]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

In [6]:
int(torch.argmax(result.logits))+1

3

In [14]:
r= requests.get("https://www.tripadvisor.com/Hotel_Review-g186338-d193102-Reviews-DoubleTree_by_Hilton_Hotel_London_West_End-London_England.html")
soup= BeautifulSoup(r.text,"html.parser")
soup

<!DOCTYPE html>
<html lang="en" xmlns:og="http://opengraphprotocol.org/schema/"><head><meta content="text/html; charset=utf-8" http-equiv="content-type"/><link href="https://static.tacdn.com/favicon.ico?v2" id="favicon" rel="icon" type="image/x-icon"/><link color="#000000" href="https://static.tacdn.com/img2/brand_refresh/application_icons/mask-icon.svg" rel="mask-icon" sizes="any"/><meta content="#34e0a1" name="theme-color"/><meta content="telephone=no" name="format-detection"/><script type="text/javascript">window.taRollupsAreAsync = true;</script><link crossorigin="" href="https://static.tacdn.com/css2/webfonts/TripSans/TripSans.css?v1.002" rel="stylesheet"/><link as="fetch" crossorigin="anonymous" href="/static/decodeKey.txt" rel="preload"><title>DOUBLETREE BY HILTON HOTEL LONDON - WEST END $153 ($̶2̶7̶7̶) - Updated 2021 Prices &amp;  Reviews - England - Tripadvisor</title><meta content="TripAdvisor" property="al:ios:app_name"/><meta content="284876795" property="al:ios:app_store_i

In [34]:

results= soup.find_all("q",class_="XllAv H4 _a")
reviews=[result.text for result in results]

In [35]:
reviews


['I stayed here for work and was not impressed. Upon check in I was given a dirty smelly room on the ground floor that had definitely seen better days. I asked to be moved and the new refurbished room was very nice, although there were dirty glasses, used water bottles and an unemptied bathroom bin - I thought rooms were thoroughly cleaned due to Covid! We used the restaurant and whilst the food was ok, our order had to be taken 3 times, took ages to arrive and came in stages! The fire alarm went off causing chaos and luckily a false alarm. To top it all I was punched b another guest whilst waiting for the lift and whilst reported to reception nothing was done. Upon check out I was asked if I’d had a nice stay, I replied no, but again nothing was done. I will be looking for an alternative for future trips. …',
 'I previously reviewed the hotel and gave it a one star due to an issue relating to a £50 additional deposit taken for incidentals. The reason and process for this has been expl

In [38]:
df= pd.DataFrame(np.array(reviews),columns=['review'])

In [40]:
df['review'][0]

'I stayed here for work and was not impressed. Upon check in I was given a dirty smelly room on the ground floor that had definitely seen better days. I asked to be moved and the new refurbished room was very nice, although there were dirty glasses, used water bottles and an unemptied bathroom bin - I thought rooms were thoroughly cleaned due to Covid! We used the restaurant and whilst the food was ok, our order had to be taken 3 times, took ages to arrive and came in stages! The fire alarm went off causing chaos and luckily a false alarm. To top it all I was punched b another guest whilst waiting for the lift and whilst reported to reception nothing was done. Upon check out I was asked if I’d had a nice stay, I replied no, but again nothing was done. I will be looking for an alternative for future trips. …'

In [41]:
def sentiment_result(x):
    tokens =tokenizer.encode(x,return_tensors='pt')
    result=model(tokens)
    return int(torch.argmax(result.logits))+1
    

In [49]:
df["sent"]=df['review'].apply(lambda x: sentiment_result(x))
                              

In [50]:
df


,review,sent
0,I stayed here for work and was not impressed. ...,2
1,I previously reviewed the hotel and gave it a ...,1
2,The hotel room was looking a bit tired. The ba...,3
3,We had afternoon tea here while watching busy ...,5
4,"Superior queen room wasVery small, not sure wh...",3


[]

In [ ]:
s